# Week 1 - Measuring Meaning & Sampling
This week, we begin by "begging, borrowing and stealing" text from several
contexts of human communication (e.g., PDFs, HTML, Word) and preparing it for
machines to "read" and analyze so that we can begin to build our sample. This notebook outlines scraping text from the web, PDF and Word documents. Then we detail "spidering" or walking
through hyperlinks to build samples of online content, and using APIs,
Application Programming Interfaces, provided by webservices to access their
content. Along the way, we will use regular expressions, outlined in the
reading, to remove unwanted formatting and ornamentation. Next, we discuss
various text encodings, filtering and data structures in which text can be
placed for analysis. Finally, we ask you to begin building a corpus for preliminary analysis and articulate what your sample represents in context of your final project. 

We made a python package just for this course: lucem_illud. If you haven't installed this package, you should run the following code first. You don't need to install the package later; all you need to do is just to import the package with: import lucem_illud. For your final projects, you may find it useful to [read the lucem_illud source code](https://github.com/UChicago-Computational-Content-Analysis/lucem_illud/tree/main/lucem_illud) and modify your code for your own interests.

In [1]:
!pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git
#installing lucem_illud package
#lucem_illud is a Latin phrase meaning "that light", the insight we can discover in text data!
#If you get an error like "Access is denied", try running the `pip` command on the command line as an administrator.

  Cloning git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git to /private/var/folders/ch/__55f9w14lv910_4yvvh38700000gn/T/pip-req-build-4gj5fv9x


If you're not familiar with jupyter notebook, you may wonder what the exclamation mark(!) at the beginning of the command does (or even what pip means). The exclamation mark enables us to execute Terminal commands in the notebook cells (e.g., run `!ls` to display files in the current folder).

For this notebook we will be using the following packages:

In [10]:
#Special module written for this class
#This provides access to data and to helper functions from previous weeks
import lucem_illud #pip install git+git://github.com/UChicago-Computational-Content-Analysis/lucem_illud.git

#All these packages need to be installed from pip
import requests #for http requests
import bs4 #called `beautifulsoup4`, an html parser
import pandas #gives us DataFrames
import docx #reading MS doc files, install as `python-docx`

#Stuff for pdfs
#Install as `pdfminer2`
import pdfminer.pdfinterp
import pdfminer.converter
import pdfminer.layout
import pdfminer.pdfpage

#These come with Python
import re #for regexs
import urllib.parse #For joining urls
import io #for making http requests look like files
import json #For Tumblr API responses
import os.path #For checking if files exist
import os #For making directories

We will also be working on the following files/urls

In [11]:
wikipedia_base_url = 'https://en.wikipedia.org'
wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
content_analysis_save = 'wikipedia_content_analysis.html'
example_text_file = 'sometextfile.txt'
information_extraction_pdf = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/Content%20Analysis%2018.pdf'
example_docx = 'https://github.com/Computational-Content-Analysis-2018/Data-Files/raw/master/1-intro/macs6000_connecting_to_midway.docx'
example_docx_save = 'example.docx'

# Scraping

Before we can start analyzing content we need to obtain it. Sometimes it will be
provided to us from a pre-curated text archive, but sometimes we will need to
download it. As a starting example we will attempt to download the wikipedia
page on content analysis. The page is located at [https://en.wikipedia.org/wiki/
Content_analysis](https://en.wikipedia.org/wiki/Content_analysis) so lets start
with that.

We can do this by making an HTTP GET request to that url, a GET request is
simply a request to the server to provide the contents given by some url. The
other request we will be using in this class is called a POST request and
requests the server to take some content we provide. While the Python standard
library does have the ability do make GET requests we will be using the
[_requests_](http://docs.python-requests.org/en/master/) package as it is _'the
only Non-GMO HTTP library for Python'_...also it provides a nicer interface.

In [12]:
#wikipedia_content_analysis = 'https://en.wikipedia.org/wiki/Content_analysis'
requests.get(wikipedia_content_analysis)

<Response [200]>

`'Response [200]'` means the server responded with what we asked for. If you get
another number (e.g. 404) it likely means there was some kind of error, these
codes are called HTTP response codes and a list of them can be found
[here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). The response
object contains all the data the server sent including the website's contents
and the HTTP header. We are interested in the contents which we can access with
the `.text` attribute.

In [13]:
wikiContentRequest = requests.get(wikipedia_content_analysis)
print(wikiContentRequest.text[:1000])

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Content analysis - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4cfd7ee7-fc80-431a-81d8-ef0d3cd5f102","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Content_analysis","wgTitle":"Content analysis","wgCurRevisionId":1065940520,"wgRevisionId":1065940520,"wgArticleId":473317,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description matches Wikidata","Articles prone to spam from October 2017","Articles with BNE identifiers",

This is not what we were looking for, because it is the start of the HTML that
makes up the website. This is HTML and is meant to be read by computers. Luckily
we have a computer to parse it for us. To do the parsing we will use [_Beautiful
Soup_](https://www.crummy.com/software/BeautifulSoup/) which is a better parser
than the one in the standard library.

But before we proceed to Beautiful Soup, a digression about Python syntax, especially about objects and functions.
For those who are not familiar with the syntax of python (or, if you're familiar with R programming), you might wonder what requests.get or wikiContentRequest.text mean. To understand this, you need to first understand what objects are. You may have heard that Python is an object oriented programming language (unlike the procedure oriented programming language, an example of which is R). Object is a set of variables (or, data) and functions into which you pass your data. So, in object oriented programming languages, like python, variables and functions are bunleded into objects.

For example, let's look at wikiContentRequest. We use dir() function, which returns the list of attributes and functions of objects.

In [14]:
 dir(wikiContentRequest)

['__attrs__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_content',
 '_content_consumed',
 '_next',
 'apparent_encoding',
 'close',
 'connection',
 'content',
 'cookies',
 'elapsed',
 'encoding',
 'headers',
 'history',
 'is_permanent_redirect',
 'is_redirect',
 'iter_content',
 'iter_lines',
 'json',
 'links',
 'next',
 'ok',
 'raise_for_status',
 'raw',
 'reason',
 'request',
 'status_code',
 'text',
 'url']

There's 'text' here. We used 'wikiContentRequest.text' to access 'text.' In other words, we use .(dot notation) to access functions from objects. wikiContentRequest has a set of functions, as shown above, and we used 'wikiContentRequest.text' to access one of them. By the way, dot notations do not necessarily refer to functions--it refers to anything that the entity contains. 



Moving on to the next step: BeautifulSoup, a Python library which extracts data from HTML and XML, and transforms HTML files into Python objects.

In [15]:
wikiContentSoup = bs4.BeautifulSoup(wikiContentRequest.text, 'html.parser')
print(wikiContentSoup.text[:200])





Content analysis - Wikipedia









































Content analysis

From Wikipedia, the free encyclopedia



Jump to navigation
Jump to search
Research method for studying docu


This is better but there's still random whitespace and we have more than just
the text of the article. This is because what we requested is the whole webpage,
not just the text for the article.

We want to extract only the text we care about, and in order to do this we will
need to inspect the html. One way to do this is simply to go to the website with
a browser and use its inspection or view source tool. If javascript or other
dynamic loading occurs on the page, however, it is likely that what Python
receives is not what you will see, so we will need to inspect what Python
receives. To do this we can save the html `requests` obtained.

In [16]:
#content_analysis_save = 'wikipedia_content_analysis.html'

with open(content_analysis_save, mode='w', encoding='utf-8') as f:
    f.write(wikiContentRequest.text)

open() is a function which literally opens and returns the file. This function has multiple modes, and, here, we used mode as 'w', which means: open a file for writing. And then, we use 'write' function to write on the empty file (content_analysis_save) that we created using open(content_analysis_save, mode='w', encoding='utf-8').} What did we write on this file? The text we got from wikiContentRequest.text

Now let's open the file (`wikipedia_content_analysis.html`) we just created with
a web browser. It should look sort of like the original but without the images
and formatting.

As there is very little standardization on structuring webpages, figuring out
how best to extract what you want is an art. Looking at this page it looks like
all the main textual content is inside `<p>`(paragraph) tags within the `<body>`
tag. 

In [17]:
contentPTags = wikiContentSoup.body.findAll('p')
for pTag in contentPTags[:3]:
    print(pTag.text)

South Asia 

Middle East

Europe



Another excursion for those who are not familiar with programming: for loop. For loop is used to iterate over a sequence. "ContentPTags" contains multiple paragraphs, each of which starts and ends with `<p>`. What the "for pTag in contentPtags[:3]" does here is: find each paragraph in contentPTags, which, here, we limited to the first three using contentPtags[:3], and then print each paragraph. So, we have three paragraphs. By the way, you can insert `<p>` in juputer notebook!

We now have all the text from the page, split up by paragraph. If we wanted to
get the section headers or references as well it would require a bit more work,
but is doable.

There is one more thing we might want to do before sending this text to be
processed, remove the references indicators (`[2]`, `[3]` , etc). To do this we
can use a short regular expression (regex).

In [18]:
contentParagraphs = []
for pTag in contentPTags:
    #strings starting with r are raw so their \'s are not modifier characters
    #If we didn't start with r the string would be: '\\[\\d+\\]'
    contentParagraphs.append(re.sub(r'\[\d+\]', '', pTag.text))

#convert to a DataFrame
contentParagraphsDF = pandas.DataFrame({'paragraph-text' : contentParagraphs})
print(contentParagraphsDF)

                                       paragraph-text
0                                       South Asia \n
1                                       Middle East\n
2                                            Europe\n
3                                     North America\n
4   Content analysis is the study of documents and...
5   Practices and philosophies of content analysis...
6   Computers are increasingly used in content ana...
7   Content analysis is best understood as a broad...
8   The simplest and most objective form of conten...
9   A further step in analysis is the distinction ...
10  Quantitative content analysis highlights frequ...
11  Siegfried Kracauer provides a critique of quan...
12  With the rise of common computing facilities l...
13  By having contents of communication available ...
14  Computer-assisted analysis can help with large...
15  Robert Weber notes: "To make valid inferences ...
16  There are five types of texts in content analy...
17  Content analysis is rese

Since we learned how to do for loop, you might get what we did here: using contentParagraphs = [], we made an empty list; and then, for each paragraph in contentPTags, we substituted every [\d+\] with '', i.e., removed every [\d+\], and then appended each paragraph (now without [\d+\]) to the empty list. As we can see, we have a dataframe, each row of which is each paragraph of contentPTags, without reference indicators. 

By the way, what does [\d+\] mean? If you are not familiar with regex, it is a way of specifying searches in text.
A regex engine takes in the search pattern, in the above case `'\[\d+\]'` and
some string, the paragraph texts. Then it reads the input string one character
at a time checking if it matches the search. Here the regex `'\d'` matches
number characters (while `'\['` and `'\]'` capture the braces on either side).

Now we have a `DataFrame` containing all relevant text from the page ready to be processed

In [19]:
findNumber = r'\d'
regexResults = re.search(findNumber, 'not a number, not a number, numbers 2134567890, not a number')
regexResults

<re.Match object; span=(36, 37), match='2'>

In Python the regex package (`re`) usually returns `Match` objects (you can have
multiple pattern hits in a a single `Match`), to get the string that matched our
pattern we can use the `.group()` method, and as we want the first one we will
ask for the 0'th group.

In [20]:
print(regexResults.group(0))

2


That gives us the first number, if we wanted the whole block of numbers we can
add a wildcard `'+'` which requests 1 or more instances of the preceding
character.

In [21]:
findNumbers = r'\d+'
regexResults = re.search(findNumbers, 'not a number, not a number, numbers 2134567890, not a number')
print(regexResults.group(0))

2134567890


Now we have the whole block of numbers, there are a huge number of special
characters in regex, for the full description of Python's implementation look at
the [re docs](https://docs.python.org/3/library/re.html) there is also a short
[tutorial](https://docs.python.org/3/howto/regex.html#regex-howto).

# <font color="red">Exercise 1</font>
<font color="red">Construct cells immediately below this that describe and download webcontent relating to your anticipated final project. Use beautiful soup and at least five regular expressions to extract relevant, nontrivial *chunks* of that content (e.g., cleaned sentences, paragraphs, etc.) to a pandas `Dataframe`.</font>

Since my proposed main text resource is Twitter, which can now only retrieved by the Twint project, I use BBC website for the web scraping excercise here. International news coverage about Taiwan can also be an significant supplemental resource, representing how international main actors see Taiwan in the global context.

Since the search page content is not enough for practicing 5 regular expressions, 3 of the regex extractions will be conducted in the news results:

##search page:
1. clean header: only keep alphabetical words in the header
2. year: retrive 4 digits from the date column

##news page:

3. extract Taiwan n-grams: retrieve up to 5-grams centering the key word "Taiwan"
4. find possessive cases: retrieve word ending with "'s" and the word after
5. find quotes/emphases: retrive words between quotation marks

In [170]:
#BBC main website search for "Taiwan"
pagenumber = 2
bbc_url = "https://www.bbc.co.uk/search?q=Taiwan&page=" + str(pagenumber)

bbc_response = requests.get(bbc_url)
bbc_Soup = bs4.BeautifulSoup(bbc_response.text, 'html.parser')

In [179]:
#Each page has 10 results--retriving the headers, subheaders, dates, and categories.

#headers (& clean headers) and URL links
bbc_headers = bbc_Soup.body.findAll('a', class_ = 'ssrcss-atl1po-PromoLink e1f5wbog0')
bbc_headers_text = []
bbc_cleanheaders_text = []
bbc_URLs = []
for header in bbc_headers:
    bbc_headers_text.append(header.text)
    bbc_URLs.append(header.get('href'))
    #regex 1
    bbc_cleanheaders_text.append(re.sub(r'[^a-zA-Z ]+', '', header.text.lower()))

#subheaders
bbc_subheaders = bbc_Soup.body.findAll('p', class_ = 'ssrcss-1q0x1qg-Paragraph eq5iqo00')
bbc_subheaders = bbc_subheaders[:10]
bbc_subheaders_text = []
for subheader in bbc_subheaders:
    bbc_subheaders_text.append(subheader.text)

#date (& year), category and subcategory
bbc_infos = bbc_Soup.body.findAll('span', class_ = 'ssrcss-8g95ls-MetadataSnippet ecn1o5v2')
bbc_dates_text = []
bbc_cats_text = []
bbc_subcats_text = []
for i in range(len(bbc_infos)):
    if i % 3 == 0:
        bbc_dates_text.append(bbc_infos[i].text)
    elif i % 3 == 1:
        bbc_cats_text.append(bbc_infos[i].text)
    else:
        bbc_subcats_text.append(bbc_infos[i].text)

bbc_years_text = []
for date in bbc_dates_text:
    #regex 2
    year = re.findall(r'\d{4}', date)
    bbc_years_text.append(year[0])

In [181]:
bbc_df = pandas.DataFrame({'header': bbc_headers_text,
                           'URL': bbc_URLs,
                           'cleanheader': bbc_cleanheaders_text,
                           'subheader': bbc_subheaders_text,
                           'date': bbc_dates_text,
                           'year': bbc_years_text,
                           'category': bbc_cats_text,
                           'subcategory': bbc_subcats_text,
                          })
bbc_df

,header,URL,cleanheader,subheader,date,year,category,subcategory
0,Newsday: Nicaragua breaks off diplomatic links...,https://www.bbc.co.uk/programmes/w172xv30tx82cmn,newsday nicaragua breaks off diplomatic links ...,It becomes the latest country to favour China ...,10 December 2021,2021,Programmes,BBC World Service
1,China downgrades diplomatic ties with Lithuani...,https://www.bbc.co.uk/news/world-asia-china-59...,china downgrades diplomatic ties with lithuani...,Lithuania allowed Taiwan to open a de facto em...,22 November 2021,2021,News,China
2,Biden-Xi talks: China warns US about 'playing ...,https://www.bbc.co.uk/news/world-asia-china-59...,bidenxi talks china warns us about playing wit...,Xi Jinping and Joe Biden hold substantial talk...,16 November 2021,2021,News,China
3,Ros Atkins onâ¦ China-Taiwan tensions,https://www.bbc.co.uk/news/world-58925971,ros atkins on chinataiwan tensions,Ros Atkins examines why tensions between China...,16 October 2021,2021,News,World
4,The Documentary Podcast: Ros Atkins on: China-...,https://www.bbc.co.uk/programmes/p09yt2gk,the documentary podcast ros atkins on chinatai...,Ros Atkins looks at whatâs behind Chinaâs ...,16 October 2021,2021,Programmes,BBC World Service
5,Ros Atkins on ...: China-Taiwan tensions,https://www.bbc.co.uk/programmes/w3ct2dnx,ros atkins on chinataiwan tensions,Ros Atkins looks at whatâs behind Chinaâs ...,16 October 2021,2021,Programmes,BBC World Service
6,Kaohsiung: Firefighters battle Taiwan tower bl...,https://www.bbc.co.uk/news/world-asia-58909907,kaohsiung firefighters battle taiwan tower blo...,At least 46 people have been killed in a fire ...,14 October 2021,2021,News,Asia
7,China-Taiwan tensions: Xi Jinping says 'reunif...,https://www.bbc.co.uk/news/world-asia-china-58...,chinataiwan tensions xi jinping says reunifica...,"Taiwan dismisses the Chinese leader's remarks,...",9 October 2021,2021,News,China
8,"Hundreds of Taiwanese extradited to China, say...",https://www.bbc.co.uk/news/world-asia-59486286,hundreds of taiwanese extradited to china says...,A human rights group says the practice is bein...,1 December 2021,2021,News,Asia
9,US deeply concerned over Taiwan-China tension,https://www.bbc.co.uk/news/world-us-canada-588...,us deeply concerned over taiwanchina tension,The US national security advisor speaks to the...,7 October 2021,2021,News,US & Canada


In [216]:
#Using a single news page for regex practice
news_url= "https://www.bbc.com/news/world-asia-china-59370521"
news_response = requests.get(news_url)
news_Soup = bs4.BeautifulSoup(news_response.text, 'html.parser')

#get paragraphs of the news article
article = news_Soup.body.findAll('article', class_ = 'ssrcss-1mc1y2-ArticleWrapper e1nh2i2l6')
paragraphs = article[0].findAll('p', class_ = 'ssrcss-1q0x1qg-Paragraph eq5iqo00')
paragraph = []
for p in paragraphs:
    paragraph.append(p.text)

In [217]:
#find Taiwan n-grams
Taiwan_ngrams = []
for p in paragraph:
    #regex 3
    Taiwan_ngrams.extend(re.findall(r'((?:\S+\s+){0,2}\bTaiwan\b\s*(?:\S+\b\s*){0,2})', p))
Taiwan_ngrams


['state allowed Taiwan to open ',
 'ties with Taiwan',
 "Taiwan's new ",
 'Beijing considers Taiwan part of ',
 'China, but Taiwan sees itself ',
 'the name Taiwan rather than ',
 '18 years. Taiwan has few ',
 'China insists Taiwan is an ',
 'engagement with Taiwan ',
 'relationship with Taiwan',
 'relations with Taiwan even amid ',
 'Taiwan currently has ',
 'ties with Taiwan']

In [218]:
#find possessive cases and the noun after
poss_cases = []
for p in paragraph:
    #regex 4
    poss_cases.extend(re.findall(r'\b\w*?\'s\b\s*(?:\S+\b\s*){0,1}', p))
poss_cases

["Taiwan's new ",
 "island's first ",
 "China's pressure",
 "China's position ",
 "China's foreign ",
 "China's sovereignty ",
 "Lithuania's prime ",
 "nation's plans ",
 "government's programme ",
 "Lithuania's move "]

In [219]:
#find words between quotation marks
quotes = []
for p in paragraph:
    #regex 5
    quotes.extend(re.findall(r'"(.*?)"', p))
quotes

['One China',
 'Chinese Taipei',
 "undermined China's sovereignty and territorial integrity",
 'bad precedent internationally',
 "Our government's programme says Lithuania wants a more intense economic, cultural and scientific relationship with Taiwan,",
 "[But] I want to emphasise that this step does not mean any conflict or disagreement with the 'One China' policy."]


# Spidering

What if we want to to get a bunch of different pages from wikipedia. We would
need to get the url for each of the pages we want. Typically, we want pages that
are linked to by other pages and so we will need to parse pages and identify the
links. Right now we will be retrieving all links in the body of the content
analysis page.

To do this we will need to find all the `<a>` (anchor) tags with `href`s
(hyperlink references) inside of `<p>` tags. `href` can have many
[different](http://stackoverflow.com/questions/4855168/what-is-href-and-why-is-
it-used) [forms](https://en.wikipedia.org/wiki/Hyperlink#Hyperlinks_in_HTML) so
dealing with them can be tricky, but generally, you will want to extract
absolute or relative links. An absolute link is one you can follow without
modification, while a relative link requires a base url that you will then
append. Wikipedia uses relative urls for its internal links: below is an example
for dealing with them.

In [245]:
#wikipedia_base_url = 'https://en.wikipedia.org'

otherPAgeURLS = []
#We also want to know where the links come from so we also will get:
#the paragraph number
#the word the link is in
for paragraphNum, pTag in enumerate(contentPTags):
    #we only want hrefs that link to wiki pages
    tagLinks = pTag.findAll('a', href=re.compile('/wiki/'), class_=False)
    for aTag in tagLinks:
        #We need to extract the url from the <a> tag
        relurl = aTag.get('href')
        linkText = aTag.text
        #wikipedia_base_url is the base we can use the urllib joining function to merge them
        #Giving a nice structured tupe like this means we can use tuple expansion later
        otherPAgeURLS.append((
            urllib.parse.urljoin(wikipedia_base_url, relurl),
            paragraphNum,
            linkText,
        ))
print(otherPAgeURLS[:10])

In [244]:
print(contentPTags)

Another excursion: Why do we use enumerate() here? enumerate() takes a collection, enumerates, and returns an enumate object with both the numbers and the collection. For example, contentPTags (the collection we used here) is comprised of paragraphs. We want the paragraph number of each paragraph. And this is what enumerate() does: it returns the paragraph number and the paragraph. 

We will be adding these new texts to our DataFrame `contentParagraphsDF` so we
will need to add 2 more columns to keep track of paragraph numbers and sources.

In [243]:
contentParagraphsDF['source'] = [wikipedia_content_analysis] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['paragraph-number'] = range(len(contentParagraphsDF['paragraph-text']))

contentParagraphsDF

Then we can add two more columns to our `Dataframe` and define a function to
parse
each linked page and add its text to our DataFrame.

In [17]:
contentParagraphsDF['source-paragraph-number'] = [None] * len(contentParagraphsDF['paragraph-text'])
contentParagraphsDF['source-paragraph-text'] = [None] * len(contentParagraphsDF['paragraph-text'])

def getTextFromWikiPage(targetURL, sourceParNum, sourceText):
    #Make a dict to store data before adding it to the DataFrame
    parsDict = {'source' : [], 'paragraph-number' : [], 'paragraph-text' : [], 'source-paragraph-number' : [],  'source-paragraph-text' : []}
    #Now we get the page
    r = requests.get(targetURL)
    soup = bs4.BeautifulSoup(r.text, 'html.parser')
    #enumerating gives use the paragraph number
    for parNum, pTag in enumerate(soup.body.findAll('p')):
        #same regex as before
        parsDict['paragraph-text'].append(re.sub(r'\[\d+\]', '', pTag.text))
        parsDict['paragraph-number'].append(parNum)
        parsDict['source'].append(targetURL)
        parsDict['source-paragraph-number'].append(sourceParNum)
        parsDict['source-paragraph-text'].append(sourceText)
    return pandas.DataFrame(parsDict)

And run it on our list of link tags

In [247]:
for urlTuple in otherPAgeURLS[:3]:
    #ignore_index means the indices will not be reset after each append
    contentParagraphsDF = contentParagraphsDF.append(getTextFromWikiPage(*urlTuple),ignore_index=True)
contentParagraphsDF


# <font color="red">Exercise 2</font>
<font color="red">Construct cells immediately below this that spider webcontent from another site with content relating to your anticipated final project. Specifically, identify urls on a core page, then follow and extract content from them into a pandas `Dataframe`. In addition, demonstrate a *recursive* spider, which follows more than one level of links (i.e., follows links from a site, then follows links on followed sites to new sites, etc.), making sure to define a reasonable endpoint so that you do not wander the web forever :-).</font>



A random BBC news page as the initial page, the spider retrieve in-paragraph links and recommendation news links of the given url until specified level.
Note that the "level" column here indicates the order of the recursive spider have went through, so the node/leaves pages are labelled as level one.

In [555]:
url = 'https://www.bbc.com/news/world-asia-china-59301167'
url_list = [url]
def spider(url_list, level, df = pandas.DataFrame(), spidered_url = set()):
    next_url_list = []
    for url in url_list:
        if url not in spidered_url and re.search(r'-\d{8}$', url) != None:
            spidered_url.add(url)
            print(url)
        
            response = requests.get(url)
            soup = bs4.BeautifulSoup(response.text, "html.parser")
    
            header = soup.findAll('h1')[0].text
            time = soup.findAll('time')[0].text
    
            article = soup.body.findAll('article', class_ = 'ssrcss-1mc1y2-ArticleWrapper e1nh2i2l6')
            if len(article) == 0:
                text = 'NA'
            else:
                paragraphs = article[0].findAll('p', class_ = 'ssrcss-1q0x1qg-Paragraph eq5iqo00')
                paragraph = []
                for p in paragraphs:
                    paragraph.append(p.text)
                text = ' '.join(paragraph)
                
                inline_urls = article[0].findAll("a", class_ = 'ssrcss-9nsdc6-InlineLink e1no5rhv0')
                promo_urls = article[0].findAll("a", class_ = 'ssrcss-o79sa2-PromoLink e1f5wbog0')
                next_url_list.extend([url['href']  for url in inline_urls])
                next_url_list.extend(["https://www.bbc.com"+url['href'] for url in promo_urls])
    
            row = pandas.DataFrame({'header' : [header],
                                    'url' : [url],
                                    'time' : [time],
                                    'text' : [text],
                                    'level' : [level]})
            df = df.append(row, ignore_index=True)

    if level == 1:
        return df, spidered_url
    
    next_level_df, next_spidered_url = spider(next_url_list, level-1, df, spidered_url)
    
    return next_level_df, next_spidered_url

In [556]:
spider(url_list, 3)[0]

https://www.bbc.com/news/world-asia-china-59301167
https://www.bbc.co.uk/news/world-asia-34729538
https://www.bbc.com/news/world-us-canada-59258375
https://www.bbc.com/news/science-environment-59238869
https://www.bbc.com/news/world-asia-china-49633862
https://www.bbc.com/news/world-asia-16464515
https://www.bbc.com/news/world-asia-china-52765838
https://www.bbc.co.uk/news/world-asia-55788359
https://www.bbc.com/news/world-asia-51077553
https://www.bbc.com/news/world-asia-54200913
https://www.bbc.com/news/world-asia-53719333
https://www.bbc.com/news/world-us-canada-53517439
https://www.bbc.com/news/world-asia-32583444
https://www.bbc.com/news/world-asia-china-34711923
https://www.bbc.co.uk/news/science-environment-59238869
https://www.bbc.co.uk/news/world-asia-59001850
https://www.bbc.co.uk/news/world-asia-china-38285354
https://www.bbc.co.uk/news/world-us-canada-56410177
https://www.bbc.com/news/world-56021205
https://www.bbc.com/news/world-asia-china-58511173
https://www.bbc.co.uk/ne

,header,url,time,text,level
0,Biden-Xi talks: China warns US about 'playing ...,https://www.bbc.com/news/world-asia-china-5930...,16 November 2021,This video can not be played Chinese President...,3
1,What's behind the China-Taiwan divide?,https://www.bbc.co.uk/news/world-asia-34729538,26 May 2021,Whether it's a bluff or a genuine threat of in...,2
2,Joe Biden and Xi Jinping: What they want from ...,https://www.bbc.com/news/world-us-canada-59258375,16 November 2021,US President Joe Biden and Chinese President X...,2
3,COP26: China and US agree to boost climate co-...,https://www.bbc.com/news/science-environment-5...,11 November 2021,China and the US have agreed to boost climate ...,2
4,Hong Kong: What is the Basic Law and how does ...,https://www.bbc.com/news/world-asia-china-4963...,20 November 2019,When Hong Kong was handed back to China on 1 J...,1
5,"Profile: Tsai Ing-wen, Taiwan's first female p...",https://www.bbc.com/news/world-asia-16464515,11 January 2020,Tsai Ing-wen made history in 2016 as Taiwan's ...,1
6,Hong Kong security law: What is it and is it w...,https://www.bbc.com/news/world-asia-china-5276...,30 June 2020,China has passed a wide-ranging new security l...,1
7,Taiwan reports 'large incursion' by Chinese wa...,https://www.bbc.co.uk/news/world-asia-55788359,25 January 2021,"Taiwan has reported a ""large incursion"" by Chi...",1
8,Taiwan election: Tsai Ing-wen wins second pres...,https://www.bbc.com/news/world-asia-51077553,11 January 2020,Taiwan's President Tsai Ing-wen has secured a ...,1
9,China holds military drill as US envoy visits ...,https://www.bbc.com/news/world-asia-54200913,18 September 2020,China says it is conducting military exercises...,1


## API (Tumblr)

Generally website owners do not like you scraping their sites. If done badly,
scarping can act like a DOS attack so you should be careful how often you make
calls to a site. Some sites want automated tools to access their data, so they
create [application programming interface
(APIs)](https://en.wikipedia.org/wiki/Application_programming_interface). An API
specifies a procedure for an application (or script) to access their data. Often
this is though a [representational state transfer
(REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) web
service, which just means if you make correctly formatted HTTP requests they
will return nicely formatted data.

A nice example for us to study is [Tumblr](https://www.tumblr.com), they have a
[simple RESTful API](https://www.tumblr.com/docs/en/api/v1) that allows you to
read posts without any complicated html parsing.

We can get the first 20 posts from a blog by making an http GET request to
`'http://{blog}.tumblr.com/api/read/json'`, were `{blog}` is the name of the
target blog. Lets try and get the posts from [http://lolcats-lol-
cat.tumblr.com/](http://lolcats-lol-cat.tumblr.com/) (Note the blog says at the
top 'One hour one pic lolcats', but the canonical name that Tumblr uses is in
the URL 'lolcats-lol-cat').

In [557]:
tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'))

print(r.text[:1000])

var tumblr_api_read = {"tumblelog":{"title":"One hour one pic lolcats","description":"","name":"lolcats-lol-cat","timezone":"Europe\/Paris","cname":false,"feeds":[]},"posts-start":0,"posts-total":3925,"posts-type":false,"posts":[{"id":"662815854023655425","url":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/662815854023655425","url-with-slug":"https:\/\/lolcats-lol-cat.tumblr.com\/post\/662815854023655425","type":"photo","date-gmt":"2021-09-20 04:00:56 GMT","date":"Mon, 20 Sep 2021 06:00:56","bookmarklet":0,"mobile":0,"feed-item":"","from-feed-id":0,"unix-timestamp":1632110456,"format":"html","reblog-key":"kjH8cg34","slug":"","is-submission":false,"like-button":"<div class=\"like_button\" data-post-id=\"662815854023655425\" data-blog-name=\"lolcats-lol-cat\" id=\"like_button_662815854023655425\"><iframe id=\"like_iframe_662815854023655425\" src=\"https:\/\/assets.tumblr.com\/assets\/html\/like_iframe.html?_v=66c22ab5319d742bca5762b8d18f9d06#name=lolcats-lol-cat&amp;post_id=66281585402365

This might not look very good on first inspection, but it has far fewer angle
braces than html, which makes it easier to parse. What we have is
[JSON](https://en.wikipedia.org/wiki/JSON) a 'human readable' text based data
transmission format based on javascript. Luckily, we can readily convert it to a
python `dict`.

In [558]:
#We need to load only the stuff between the curly braces
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
print(d.keys())
print(len(d['posts']))

dict_keys(['tumblelog', 'posts-start', 'posts-total', 'posts-type', 'posts'])
20


If we read the [API specification](https://www.tumblr.com/docs/en/api/v1), we
will see there are a lot of things we can get if we add things to our GET
request. First we can retrieve posts by their id number. Let's first get post
`146020177084`.

In [559]:
r = requests.get(tumblrAPItarget.format('lolcats-lol-cat'), params = {'id' : 146020177084})
d = json.loads(r.text[len('var tumblr_api_read = '):-2])
d['posts'][0].keys()
d['posts'][0]['photo-url-1280']

with open('lolcat.gif', 'wb') as f:
    gifRequest = requests.get(d['posts'][0]['photo-url-1280'], stream = True)
    f.write(gifRequest.content)

<img src='lolcat.gif'>

Such beauty; such vigor (If you can't see it you have to refresh the page). Now
we could retrieve the text from all posts as well
as related metadata, like the post date, caption or tags. We could also get
links to all the images.

In [560]:
#Putting a max in case the blog has millions of images
#The given max will be rounded up to the nearest multiple of 50
def tumblrImageScrape(blogName, maxImages = 200):
    #Restating this here so the function isn't dependent on any external variables
    tumblrAPItarget = 'http://{}.tumblr.com/api/read/json'

    #There are a bunch of possible locations for the photo url
    possiblePhotoSuffixes = [1280, 500, 400, 250, 100]

    #These are the pieces of information we will be gathering,
    #at the end we will convert this to a DataFrame.
    #There are a few other datums we could gather like the captions
    #you can read the Tumblr documentation to learn how to get them
    #https://www.tumblr.com/docs/en/api/v1
    postsData = {
        'id' : [],
        'photo-url' : [],
        'date' : [],
        'tags' : [],
        'photo-type' : []
    }

    #Tumblr limits us to a max of 50 posts per request
    for requestNum in range(maxImages // 50):
        requestParams = {
            'start' : requestNum * 50,
            'num' : 50,
            'type' : 'photo'
        }
        r = requests.get(tumblrAPItarget.format(blogName), params = requestParams)
        requestDict = json.loads(r.text[len('var tumblr_api_read = '):-2])
        for postDict in requestDict['posts']:
            #We are dealing with uncleaned data, we can't trust it.
            #Specifically, not all posts are guaranteed to have the fields we want
            try:
                postsData['id'].append(postDict['id'])
                postsData['date'].append(postDict['date'])
                postsData['tags'].append(postDict['tags'])
            except KeyError as e:
                raise KeyError("Post {} from {} is missing: {}".format(postDict['id'], blogName, e))

            foundSuffix = False
            for suffix in possiblePhotoSuffixes:
                try:
                    photoURL = postDict['photo-url-{}'.format(suffix)]
                    postsData['photo-url'].append(photoURL)
                    postsData['photo-type'].append(photoURL.split('.')[-1])
                    foundSuffix = True
                    break
                except KeyError:
                    pass
            if not foundSuffix:
                #Make sure your error messages are useful
                #You will be one of the users
                raise KeyError("Post {} from {} is missing a photo url".format(postDict['id'], blogName))

    return pandas.DataFrame(postsData)
tumblrImageScrape('lolcats-lol-cat', 50)

,id,photo-url,date,tags,photo-type
0,662815854023655425,https://64.media.tumblr.com/021eac8fbcafbb00a5...,"Mon, 20 Sep 2021 06:00:56","[gif, lolcat, lolcats, cat, funny]",gif
1,662778109891952640,https://64.media.tumblr.com/8c0517adb8c71e4a3d...,"Sun, 19 Sep 2021 20:01:00","[cat, cats, lol, lolcat, lolcats]",png
2,662657302700146688,https://64.media.tumblr.com/061d27cda309d5c809...,"Sat, 18 Sep 2021 12:00:50","[cat, cats, lol, lolcat, lolcats]",jpg
3,662513901538246656,https://64.media.tumblr.com/80584a9d1ff4ddc4fc...,"Thu, 16 Sep 2021 22:01:32","[cat, cats, lol, lolcat, lolcats]",png
4,662257177983090688,https://64.media.tumblr.com/893b320cd2e8970a20...,"Tue, 14 Sep 2021 02:01:01","[cat, cats, lol, lolcat, lolcats]",jpg
5,662166591527698432,https://64.media.tumblr.com/c7f0a0a9184e480e15...,"Mon, 13 Sep 2021 02:01:11","[cat, cats, lol, lolcat, lolcats]",jpg
6,662113740899090432,https://64.media.tumblr.com/07f7be7f71917a6049...,"Sun, 12 Sep 2021 12:01:09","[cat, cats, lol, lolcat, lolcats]",jpg
7,661955166248026112,https://64.media.tumblr.com/205f030c48d31f8960...,"Fri, 10 Sep 2021 18:00:40","[cat, cats, lol, lolcat, lolcats]",jpg
8,661894830378614784,https://64.media.tumblr.com/c463bff883fec2045b...,"Fri, 10 Sep 2021 02:01:39","[cat, cats, lol, lolcat, lolcats]",png
9,661864596682964992,https://64.media.tumblr.com/b840ddb3aa2303206b...,"Thu, 09 Sep 2021 18:01:06","[cat, cats, lol, lolcat, lolcats]",jpg


Now we have the urls of a bunch of images and can run OCR on them to gather
compelling meme narratives, accompanied by cats.

# Files

What if the text we want isn't on a webpage? There are a many other sources of
text available, typically organized into *files*.

## Raw text (and encoding)

The most basic form of storing text is as a _raw text_ document. Source code
(`.py`, `.r`, etc) is usually raw text as are text files (`.txt`) and those with
many other extension (e.g., .csv, .dat, etc.). Opening an unknown file with a
text editor is often a great way of learning what the file is.

We can create a text file in python with the `open()` function

In [561]:
#example_text_file = 'sometextfile.txt'
#stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols \u2421 \u241B \u20A0 \u20A1 \u20A2 \u20A3 \u0D60\n'
stringToWrite = 'A line\nAnother line\nA line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ\n'

with open(example_text_file, mode = 'w', encoding='utf-8') as f:
    f.write(stringToWrite)

Notice the `encoding='utf-8'` argument, which specifies how we map the bits from
the file to the glyphs (and whitespace characters like tab (`'\t'`) or newline
(`'\n'`)) on the screen. When dealing only with latin letters, arabic numerals
and the other symbols on America keyboards you usually do not have to worry
about encodings as the ones used today are backwards compatible with
[ASCII](https://en.wikipedia.org/wiki/ASCII), which gives the binary
representation of 128 characters.

Some of you, however, will want to use other characters (e.g., Chinese
characters). To solve this there is
[Unicode](https://en.wikipedia.org/wiki/Unicode) which assigns numbers to
symbols, e.g., 041 is `'A'` and 03A3 is `'Σ'` (numbers starting with 0 are
hexadecimal). Often non/beyond-ASCII characters are called Unicode characters.
Unicode contains 1,114,112 characters, about 10\% of which have been assigned.
Unfortunately there are many ways used to map combinations of bits to Unicode
symbols. The ones you are likely to encounter are called by Python _utf-8_,
_utf-16_ and _latin-1_. _utf-8_ is the standard for Linux and Mac OS while both
_utf-16_ and _latin-1_ are used by windows. If you use the wrong encoding,
characters can appear wrong, sometimes change in number or Python could raise an
exception. Lets see what happens when we open the file we just created with
different encodings.

In [562]:
with open(example_text_file, encoding='utf-8') as f:
    print("This is with the correct encoding:")
    print(f.read())

with open(example_text_file, encoding='latin-1') as f:
    print("This is with the wrong encoding:")
    print(f.read())

This is with the correct encoding:
A line
Another line
A line with a few unusual symbols ␡ ␛ ₠ ₡ ₢ ₣ ൠ

This is with the wrong encoding:
A line
Another line
A line with a few unusual symbols â¡ â â  â¡ â¢ â£ àµ 



Notice that with _latin-1_ the unicode characters are mixed up and there are too
many of them. You need to keep in mind encoding when obtaining text files.
Determining the encoding can sometime involve substantial work.

We can also load many text files at once. Lets start by looking at the Shakespeare files in the `data` directory 

In [564]:
with open('./data/Shakespeare/midsummer_nights_dream.txt') as f:
    midsummer = f.read()
print(midsummer[-700:])

By the way, depending on your working directory, you might get errors such as: [Errno 2] No such file or directory: '../data/Shakespeare/midsummer_nights_dream.txt.' Don't panic, it's nothing, just check your working directory. 

Then to load all the files in `./data/Shakespeare` we can use a for loop with `scandir`:

In [ ]:
targetDir = './data/Shakespeare' #Change this to your own directory of texts
shakespearText = []
shakespearFileName = []

for file in (file for file in os.scandir(targetDir) if file.is_file() and not file.name.startswith('.')):
    with open(file.path, encoding="utf-8") as f:
        shakespearText.append(f.read())
    shakespearFileName.append(file.name)

Then we can put them all in pandas DataFrame

In [ ]:
shakespear_df = pandas.DataFrame({'text' : shakespearText}, index = shakespearFileName)
shakespear_df

Getting your text in a format like this is the first step of most analysis

## PDF

Another common way text will be stored is in a PDF file. First we will download
a pdf in Python. To do that lets grab a chapter from
_Speech and Language Processing_, chapter 21 is on Information Extraction which
seems apt. It is stored as a pdf at [https://web.stanford.edu/~jurafsky/slp3/21.
pdf](https://web.stanford.edu/~jurafsky/slp3/21.pdf) although we are downloading
from a copy just in case Jurafsky changes their website.

In [566]:
information_extraction_pdf = 'https://github.com/KnowledgeLab/content_analysis/raw/data/21.pdf'

infoExtractionRequest = requests.get(information_extraction_pdf, stream=True)
print(infoExtractionRequest.text[:1000])

%PDF-1.5
%����
97 0 obj
<<
/Length 2988      
/Filter /FlateDecode
>>
stream
xڝYY��~�_��T,U�����f<{����z\[)o(
���!�������n��4�J�/b�� }7�9�s������o���I�,�|�a�d�J��M��y�9?�~<i]כHE���	�}����e��k��esp�ߋ�շyS�Jf��|�?>���׺�+��y7����==w��8�����M<�x�
߉�M�Q�k�k�tW�C�6덂
������拍���Y��o乤�8s���Fy-�����g��vGvM ����3���}���[+o�y��שnٰ|7N6UŽ�f����l����D�D��|[i�n��MY��f�3��W�n`�>���X!~lڧ��>��K�7�
��P�z������s;
�zx�����t��|ӎ���n���V{�Av@�Wt:�!bc!*���>����΋�@R�Q`��_�A���Aл�yG+J���z[��\[G�1-r��U�m�A�|��'�2


It says `'pdf'`, so thats a good sign. The rest though looks like we are having
issues with an encoding. The random characters are not caused by our encoding
being wrong, however. They are cause by there not being an encoding for those
parts at all. PDFs are nominally binary files, meaning there are sections of
binary that are specific to pdf and nothing else so you need something that
knows about pdf to read them. To do that we will be using
[`PyPDF2`](https://github.com/mstamy2/PyPDF2), a PDF processing library for
Python 3.


Because PDFs are a very complicated file format pdfminer requires a large amount
of boilerplate code to extract text, we have written a function that takes in an
open PDF file and returns the text so you don't have to.

In [567]:
def readPDF(pdfFile):
    #Based on code from http://stackoverflow.com/a/20905381/4955164
    #Using utf-8, if there are a bunch of random symbols try changing this
    codec = 'utf-8'
    rsrcmgr = pdfminer.pdfinterp.PDFResourceManager()
    retstr = io.StringIO()
    layoutParams = pdfminer.layout.LAParams()
    device = pdfminer.converter.TextConverter(rsrcmgr, retstr, laparams = layoutParams, codec = codec)
    #We need a device and an interpreter
    interpreter = pdfminer.pdfinterp.PDFPageInterpreter(rsrcmgr, device)
    password = ''
    maxpages = 0
    caching = True
    pagenos=set()
    for page in pdfminer.pdfpage.PDFPage.get_pages(pdfFile, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
    device.close()
    returnedString = retstr.getvalue()
    retstr.close()
    return returnedString

First we need to take the response object and convert it into a 'file like'
object so that pdfminer can read it. To do this we will use `io`'s `BytesIO`.

In [568]:
infoExtractionBytes = io.BytesIO(infoExtractionRequest.content)

Now we can give it to pdfminer.

In [569]:
print(readPDF(infoExtractionBytes)[:550])

Speech and Language Processing. Daniel Jurafsky & James H. Martin.
rights reserved.

Draft of November 7, 2016.

Copyright c(cid:13) 2016.

All

CHAPTER

21 Information Extraction

I am the very model of a modern Major-General,
I’ve information vegetable, animal, and mineral,
I know the kings of England, and I quote the ﬁghts historical
From Marathon to Waterloo, in order categorical...
Gilbert and Sullivan, Pirates of Penzance

Imagine that you are an analyst with an investment ﬁrm that tracks airline stocks.
You’re given the task of determini


From here we can either look at the full text or fiddle with our PDF reader and
get more information about individual blocks of text.

## Word Docs

The other type of document you are likely to encounter is the `.docx`, these are
actually a version of [XML](https://en.wikipedia.org/wiki/Office_Open_XML), just
like HTML, and like HTML we will use a specialized parser.

For this class we will use [`python-docx`](https://python-
docx.readthedocs.io/en/latest/) which provides a nice simple interface for
reading `.docx` files

In [ ]:
#example_docx = 'https://github.com/KnowledgeLab/content_analysis/raw/data/example_doc.docx'

r = requests.get(example_docx, stream=True)
d = docx.Document(io.BytesIO(r.content))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

This procedure uses the `io.BytesIO` class again, since `docx.Document` expects
a file. Another way to do it is to save the document to a file and then read it
like any other file. If we do this we can either delete the file afterwords, or
save it and avoid downloading the following time.

This function is useful as a part of many different tasks so it and others like it will be added to the helper package `lucem_illud` so we can use it later without having to retype it.

In [ ]:
def downloadIfNeeded(targetURL, outputFile, **openkwargs):
    if not os.path.isfile(outputFile):
        outputDir = os.path.dirname(outputFile)
        #This function is a more general os.mkdir()
        if len(outputDir) > 0:
            os.makedirs(outputDir, exist_ok = True)
        r = requests.get(targetURL, stream=True)
        #Using a closure like this is generally better than having to
        #remember to close the file. There are ways to make this function
        #work as a closure too
        with open(outputFile, 'wb') as f:
            f.write(r.content)
    return open(outputFile, **openkwargs)

This function will download, save and open `outputFile` as `outputFile` or just
open it if `outputFile` exists. By default `open()` will open the file as read
only text with the local encoding, which may cause issues if its not a text
file.

In [ ]:
try:
    d = docx.Document(downloadIfNeeded(example_docx, example_docx_save))
except Exception as e:
    print(e)

We need to tell `open()` to read in binary mode (`'rb'`), this is why we added
`**openkwargs`, this allows us to pass any keyword arguments (kwargs) from
`downloadIfNeeded` to `open()`.

In [ ]:
d = docx.Document(downloadIfNeeded(example_docx, example_docx_save, mode = 'rb'))
for paragraph in d.paragraphs[:7]:
    print(paragraph.text)

Now we can read the file with `docx.Document` and not have to wait for it to be
downloaded every time.

# <font color="red">Exercise 3</font>
<font color="red">Construct cells immediately below this that extract and organize textual content from text, PDF or Word into a pandas dataframe.</font>


I use a unscrapable pdf link in Exercise 2 (found in one of the level-branches). It is actually a survey, and I only kept the questions in the dataframe for practice. (items without a question such as age and occupation are excluded)

In [576]:
headers = {"User-Agent" : "web scraper for class fyeh@uchicago.edu"}

pdf = 'https://ws.mac.gov.tw/001/Upload/297/relfile/8010/6074/46a46ca3-97ed-4224-8f3d-531125d758ac.pdf'
pdf_response = requests.get(pdf, stream=True, params = headers)

pdf_Bytes = io.BytesIO(pdf_response.content)
pdf_text = readPDF(pdf_Bytes)
print(pdf_text[:550])

Percentage Distribution of the Questionnaire for 
the Survey on "Public Views on Current Cross-

Strait Issues"* 

 

Survey Date: March 19 to 23, 2021 

Number of Effective Samples: 1,078 

Sampling Error: ± 2.99 

Survey Commissioned by: Mainland Affairs Council 

Survey Unit: Election Study Center of National Chengchi University 

 

1. Do you think that the mainland Chinese government's attitude toward the ROC 
government is friendly or unfriendly? 

Extremely 

friendly 

Friendly 

Unfriendly 

Extremely 
unfriendly 

Don't know / 
No opi


In [670]:
question_lst = re.findall(r'\d{1,2}\.\s[A-Z]\w+\s+.+', pdf_text)
question_lst = [re.sub(r'\n\n.+', '', q) for q in question_lst]
q_df = pandas.DataFrame({"question" : question_lst})
q_df

,question
0,1. Do you think that the mainland Chinese gove...
1,2. Do you think that the mainland Chinese gove...
2,3. Do you feel that the current pace of cross-...
3,4. Regarding relations between Taiwan and main...
4,"5. The CCP proposed the ""one country, two syst..."
5,6. The government emphasized that Taiwan's fut...
6,"7. The President emphasized that, as long as t..."
7,"8. The President emphasized that, when the pan..."
8,9. The government has a positive outlook for c...
9,10. To maintain the order of cross-Strait exch...


# <font color="red">Exercise 4</font>

<font color="red">In the two cells immediately following, describe a possible project (e.g., it might end up being your final project, but need not be if you are still searching): **WHAT** you will analyze--the texts you will select and the social game, world and actors you intend to learn about through your analysis (<100 words); **WHY** you will analyze these texts to learn about that context--justify the rationale behind your proposed sample design for this project, based on the readings. What is the social game, social work, or social actors about whom you are seeking to make inferences? What are the virtues of your proposed sample with respect to your research questions? What are its limitations? What are alternatives? What would be a reasonable path to "scale up" your sample for further analysis (i.e., high-profile publication)? (<150 words)? [**Note**: your individual or collective project will change over the course of the quarter as new data and/or analysis opportunities arise or if old ones fade away.] 

## ***What?*** 
<100 words

My proposed topic is to measure Taiwanese Nationalism on Twitter. Since Twitter is the main realm of international propaganda for Taiwanese identity, retweets under the posts of the political opinion leaders can represent the bipolarity of pro/against independence voices. Through retrieving the Tweets relevant Taipei-Beijing tension, it is possible to quantify and analyze the fluctuation of the "imagined community" of Taiwanese people. In this social game of conveying the legitimacy of “Taiwan as a state”,  the wordings and expressions of the unorganized, self-motivated group of netizens have shown the aggregate (yet possibly diversified) “Taiwanese” image to the English world. 

## ***Why?***
<150 words

To comment about Taiwan’s status on Twitter requires strong incentives from the actors, since Twitter is not the main social media website in Taiwan, but rather an outreaching channel; they are possibly mobilized by certain international incidents, believing that they can play a part in the game fighting for the Taiwanese narrative. Tweets of Tsai Ing-wen, Lijian Zhao, and Hua Chunying become the frontlines of the pro/anti-PRC battlefield, the sharp confrontation between the two propagandas can also create echo chambers boosting inner confidence. To clearly capture the ideologies within the short contents, the inferences should be carefully made to avoid biases. The international voices from global powers and leading media would also affect the narratives by adding multiple issue frameworks to the cross-strait relationship: such as democracy backsliding, human rights protection, pandemic governance, and trade disputes. Thus, studying the contents of the third-party actors can provide a more comprehensive understanding of online ideological warfare.

### Sources:

Other popular sources for internet data:

[reddit](https://www.reddit.com/) - https://praw.readthedocs.io/en/v2.1.21/

[twitter](https://twitter.com/) - https://pypi.org/project/python-twitter/

[project gutenburg](https://www.gutenberg.org/) - https://github.com/ageitgey/Gutenberg 

